In [7]:
pip install atari_py-1.2.2-cp38-cp38-win_amd64.whl

Processing c:\users\dell\downloads\atari_py-1.2.2-cp38-cp38-win_amd64.whl
atari-py is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install gym==0.19.0

  Using cached gym-0.19.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install piglet

  Using cached piglet-1.0.0-py2.py3-none-any.whl (2.2 kB)
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install pyglet

  Using cached pyglet-1.5.21-py3-none-any.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
!pip install --user tensorflow==2.3.1 gym keras-rl2 gym[atari]

  Using cached atari_py-0.2.6-cp38-cp38-win_amd64.whl


Reason for being yanked: re-release with new wheels


In [2]:
import gym 
import random

Creating an environment

In [3]:
env = gym.make('Asterix-v0')
height, width, channels = env.observation_space.shape
actions = env.action_space.n

Actions

In [4]:
env.unwrapped.get_action_meanings()

['NOOP',
 'UP',
 'RIGHT',
 'LEFT',
 'DOWN',
 'UPRIGHT',
 'UPLEFT',
 'DOWNRIGHT',
 'DOWNLEFT']

Now let us run some episodes with random moves

In [11]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:150.0
Episode:2 Score:350.0
Episode:3 Score:300.0
Episode:4 Score:250.0
Episode:5 Score:250.0


Creating a deep learning model

In [12]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

In [13]:
def new_model(height, width, channels, actions):
    mod = Sequential()
    mod.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    mod.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    mod.add(Convolution2D(64, (3,3), activation='relu'))
    mod.add(Flatten())
    mod.add(Dense(512, activation='relu'))
    mod.add(Dense(256, activation='relu'))
    mod.add(Dense(actions, activation='linear'))
    return mod

In [20]:
del mod

In [21]:
mod = new_model(height, width, channels, actions)

In [22]:
mod.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 67584)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               34603520  
_________________________________________________________________
dense_8 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_9 (Dense)              (None, 9)                

Building an agent

In [14]:
pip install keras-rl

Note: you may need to restart the kernel to use updated packages.


In [23]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

In [24]:
def new_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn_agent = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn_agent

In [26]:
dqn_agent = new_agent(mod, actions)
dqn_agent.compile(Adam(lr=1e-4))

In [28]:
dqn_agent.fit(env, nb_steps=5000, visualize=False, verbose=2)

Training for 5000 steps ...
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
  422/5000: episode: 1, duration: 13.361s, episode steps: 422, steps per second:  32, episode reward: 250.000, mean reward:  0.592 [ 0.000, 50.000], mean action: 3.898 [0.000, 8.000],  loss: --, mean_q: --, mean_eps: --
  882/5000: episode: 2, duration: 14.131s, episode steps: 460, steps per second:  33, episode reward: 300.000, mean reward:  0.652 [ 0.000, 50.000], mean action: 3.865 [0.000, 8.000],  loss: --, mean_q: --, mean_eps: --
 1170/5000: episode: 3, duration: 217.315s, episode steps: 288, steps per second:   1, episode reward: 150.000, mean reward:  0.521 [ 0.000, 50.000], mean action: 4.066 [0.000, 8.000],  loss: 95.556266, mean_q: 37.515407, mean_eps: 0.902350
 1467/5000: episode: 4, duration: 392.690s, episode steps: 297, steps per second:   1, episode reward: 200.000, mean reward:  0.673 [ 0.000, 50.000], mean action: 3.976 [0.00

In [29]:
reward = dqn_agent.test(env, nb_episodes=10, visualize=True)
print(np.mean(reward.history['episode_reward']))  

Testing for 10 episodes ...
Episode 1: reward: 150.000, steps: 249
Episode 2: reward: 300.000, steps: 329
Episode 3: reward: 150.000, steps: 259
Episode 4: reward: 150.000, steps: 362
Episode 5: reward: 500.000, steps: 468
Episode 6: reward: 300.000, steps: 441
Episode 7: reward: 150.000, steps: 313
Episode 8: reward: 400.000, steps: 335
Episode 9: reward: 250.000, steps: 361
Episode 10: reward: 400.000, steps: 361
275.0


In [31]:
dqn_agent.save_weights('D:\Aberdeen\Applied Artificial Intelligence\weightsave5000')
dqn_agent.load_weights('D:\Aberdeen\Applied Artificial Intelligence\weightsave5000')

[WARNING] D:\Aberdeen\Applied Artificial Intelligence\weightsave5000.index already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True!


Referemces:
    https://github.com/nicknochnack/KerasRL-OpenAI-Atari-SpaceInvadersv0